In [ ]:
pip install tensorflow==2.7.0  
pip install keras==2.7.0 
pip install scikit-image==0.22.0 
pip install scikit-learn==1.3.2 

In [ ]:
import os
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from datetime import datetime 
import cv2
from PIL import Image
from keras import backend, optimizers
import glob
from tensorflow.keras.models import save_model
import pickle
from models import Attention_UNet, dice_coef, dice_coef_loss, jacard_coef

In [ ]:
seed = 42
np.random.seed = seed

IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 1

In [ ]:
image_paths = sorted(glob.glob("C:/Users/drsaq/OneDrive/Desktop/256_paches/train/images/*.jpg"))
mask_paths = sorted(glob.glob("C:/Users/drsaq/OneDrive/Desktop/256_paches/train/masks/*.jpg"))

# Initialize lists to store the images and masks
large_images = []
large_masks = []

# Iterate through the sorted file paths and load images and masks
for img_path, mask_path in zip(image_paths, mask_paths):
    # Read and process the image
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img = np.expand_dims(img, axis=-1)
    large_images.append(img)

    # Read and process the mask
    mask = cv2.imread(mask_path, 0)
    mask = np.expand_dims(mask, axis=-1)
    mask = np.where(mask < 100, 0, 255)

    # Convert the mask to boolean data type
    mask = mask.astype(bool)
    large_masks.append(mask)

# Convert the lists to NumPy arrays
X_train = np.array(large_images)/255.
y_train = np.expand_dims((np.array(large_masks)),3) /255.

image_paths = sorted(glob.glob("C:/Users/drsaq/OneDrive/Desktop/256_paches/val/images/*.jpg"))
mask_paths = sorted(glob.glob("C:/Users/drsaq/OneDrive/Desktop/256_paches/val/masks/*.jpg"))

# Initialize lists to store the images and masks
large_images = []
large_masks = []

# Iterate through the sorted file paths and load images and masks
for img_path, mask_path in zip(image_paths, mask_paths):
    # Read and process the image
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img = np.expand_dims(img, axis=-1)
    large_images.append(img)

    # Read and process the mask
    mask = cv2.imread(mask_path, 0)
    mask = np.expand_dims(mask, axis=-1)
    mask = np.where(mask < 100, 0, 255)

    # Convert the mask to boolean data type
    mask = mask.astype(bool)
    large_masks.append(mask)

# Convert the lists to NumPy arrays
X_test = np.array(large_images)/255.
y_test= np.expand_dims((np.array(large_masks)),3) /255.

In [ ]:
IMG_HEIGHT = X_train.shape[1]
IMG_WIDTH  = X_train.shape[2]
IMG_CHANNELS = X_train.shape[3]
num_labels = 1  #Binary
input_shape = (IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS)
batch_size = 1

In [ ]:
'''
Attention UNet
'''
att_unet_model = Attention_UNet(input_shape)

att_unet_model.compile(optimizer=Adam(lr = 1e-2), loss='binary_crossentropy', 
              metrics=['accuracy', jacard_coef])

save_model(att_unet_model, 'C:/Users/drsaq/OneDrive/Desktop/256_paches/Result/spore_Attention_UNet_model_architecture.h5')
print(att_unet_model.summary())
start2 = datetime.now() 
att_unet_history = att_unet_model.fit(X_train, y_train, 
                    verbose=1,
                    batch_size = batch_size,
                    validation_data=(X_test, y_test ), 
                    shuffle=False,
                    epochs=50)

with open('C:/Users/drsaq/OneDrive/Desktop/256_paches/Result/spore_Attention_UNet_model_history.pkl', 'wb') as file:
    pickle.dump(att_unet_history.history, file)
stop2 = datetime.now()
#Execution time of the model 
execution_time_Att_Unet = stop2-start2
print("Attention UNet execution time is: ", execution_time_Att_Unet)

att_unet_model.save('C:/Users/drsaq/OneDrive/Desktop/256_paches/Result/spore_Attention_UNet_50epochs_CrossEntropy.hdf5')


